# About
* **Author**: Adil Rashitov (adil.rashitov.98@gmail.com)
* **Created at**: 08.12.2022


In [ ]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Visualization libraries
# import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Configure project PATH
from pathlib import Path
import sys
PROJECT_PATH = os.getcwd().rsplit('/', 1)[0]


if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

# Data

In [ ]:
gps_records = pd.read_parquet("../data/02_intermediate/gps_records.parquet")
# route_plans = pd.read_parquet("../data/02_intermediate/route_plan.parquet")

In [ ]:
partitons = {}
for partition_key in gps_records["route_id"].unique():
    partitons[partition_key]

In [ ]:
gps_records.to_csv(
    path="../data/03_primary/gps_samples/",
    index=False,
    partition_cols=["route_id"],
)

# Main

In [ ]:
# VFHDBSCAN
from gps_activity import ActivityExtractionSession
from gps_activity.extraction.factory.preprocessing import PreprocessingFactory
from gps_activity.extraction.factory.fragmentation import VelocityFragmentationFactory
from gps_activity.extraction.factory.clustering import FDBSCANFactory


preprocessing = PreprocessingFactory.factory_pipeline(
    source_lat_column="lat",
    source_lon_column="lon",
    source_datetime="datetime",
    source_vehicle_id="plate_no",
    source_crs="EPSG:4326",
    target_crs="EPSG:2326",
)

fragmentation = VelocityFragmentationFactory.factory_pipeline(max_velocity_hard_limit=4)
clustering = FDBSCANFactory.factory_pipeline(
    source_vehicle_id_column="plate_no",
    eps=30,
    min_samples=3,
)

vfhdbscan = ActivityExtractionSession(
    preprocessing=preprocessing,
    fragmentation=fragmentation,
    clustering=clustering,
)

In [ ]:
# STCM
from gps_activity import ActivityExtractionSession
from gps_activity.extraction.factory.preprocessing import PreprocessingFactory
from gps_activity.extraction.factory.fragmentation import VelocityFragmentationFactory
from gps_activity.extraction.factory.clustering import STCMFactory


preprocessing = PreprocessingFactory.factory_pipeline(
    source_lat_column="lat",
    source_lon_column="lon",
    source_datetime="datetime",
    source_vehicle_id="plate_no",
    source_crs="EPSG:4326",
    target_crs="EPSG:2326",
)

fragmentation = VelocityFragmentationFactory.factory_pipeline(max_velocity_hard_limit=4)
clustering = STCMFactory.factory_pipeline(
    source_vehicle_id_column="plate_no",
    eps=30,
    min_duration_sec=60
)

stcm = ActivityExtractionSession(
    preprocessing=preprocessing,
    fragmentation=fragmentation,
    clustering=clustering,
)

In [ ]:
def split_gps_over_vehicle_id(gps_records: pd.DataFrame):
    gps = []
    for vehicle_ids in gps_records["plate_no"].unique():
        gps_slice = gps_records[gps_records["plate_no"] == vehicle_ids].reset_index(drop=True)
        gps.append(gps_slice.copy())
    return gps


clustered_gps_stcm = [ stcm.predict(gps) for gps in split_gps_over_vehicle_id(gps_records.copy()) ]
clustered_gps_vfhdbscan = [ stcm.predict(gps) for gps in split_gps_over_vehicle_id(gps_records.copy()) ]

In [ ]:
gps = pd.read_csv("../gps_records1.csv")
gps.head(1)

In [ ]:
import pandera as pa


columns_schema = {
    "datetime": pa.Column(
        dtype=str,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "plate_no": pa.Column(
        dtype=str,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "lat": pa.Column(
        dtype=float,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "lon": pa.Column(
        dtype=float,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "computed_velocity": pa.Column(
        dtype=float,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "cluster_id": pa.Column(
        dtype=int,
        nullable=False,
        required=True,
        coerce=True,
        unique=False,
    ),
    "lat_centroid": pa.Column(
        dtype=float,
        nullable=True,
        required=True,
        coerce=True,
        unique=False,
    ),
    "lon_centroid": pa.Column(
        dtype=float,
        nullable=True,
        required=True,
        coerce=True,
        unique=False,
    ),
    "service_start_time": pa.Column(
        dtype=str,
        nullable=True,
        required=True,
        coerce=True,
        unique=False,
    ),
    "service_end_time": pa.Column(
        dtype=str,
        nullable=True,
        required=True,
        coerce=True,
        unique=False,
    ),
    "service_duration": pa.Column(
        dtype=float,
        nullable=True,
        required=True,
        coerce=True,
        unique=False,
    ),
}

schema = pa.DataFrameSchema(
    columns=columns_schema,
    coerce=True,
    description="Data provided from customer excel file",
)


def validate_visualization_schema(gps: pd.DataFrame) -> pd.DataFrame:
    gps[["service_start_time", "service_end_time"]] = \
        gps[["service_start_time", "service_end_time"]].fillna("")
    gps = schema.validate(gps)
    gps = gps.loc[:, list(schema.columns.keys())]
    return gps


gps_to_viz = validate_visualization_schema(gps.copy())

from keplergl import KeplerGl


kepler_map = KeplerGl(data={"gps": gps}, height=900)
kepler_map

In [ ]:
import json
with open('../config/kepler_map_configuration.json', 'w', encoding='utf-8') as f:
    json.dump(kepler_map.config, f, ensure_ascii=False, indent=4)


In [ ]:
kepler_map._repr_html_(read_only=True)